# 手書き文字評価システム - リファクタリング実験

このノートブックでは、`test.py` を `src/eval` ディレクトリ構造にリファクタリングする過程を記録し、
パラメータチューニングや実験を行います。

## 目標
- 既存の `test.py` を適切なモジュール構成に分割
- 各評価軸（形・黒・白・場）のパラメータ調整
- 新しいモジュール構成での動作確認

## 1. 既存コードの確認

まず、現在の `test.py` の内容を確認し、どのような関数が含まれているかを整理します。

In [ ]:
import sys
sys.path.append('/workspace')

# 既存のtest.pyを読み込んで確認
with open('/workspace/test.py', 'r', encoding='utf-8') as f:
    test_py_content = f.read()

print("=== test.py の関数一覧 ===")
import re
functions = re.findall(r'^def (\w+)\(', test_py_content, re.MULTILINE)
for func in functions:
    print(f"- {func}()")

print(f"\n総行数: {len(test_py_content.splitlines())} 行")

## 2. ディレクトリ構成の確認

新しいディレクトリ構成が正しく作成されているかを確認します。

In [ ]:
import os
from pathlib import Path

# ディレクトリ構成を表示
def show_tree(path, prefix="", max_depth=3, current_depth=0):
    if current_depth >= max_depth:
        return
    
    path = Path(path)
    items = sorted([p for p in path.iterdir() if not p.name.startswith('.')])
    
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{item.name}")
        
        if item.is_dir() and current_depth < max_depth - 1:
            next_prefix = prefix + ("    " if is_last else "│   ")
            show_tree(item, next_prefix, max_depth, current_depth + 1)

print("=== プロジェクト構成 ===")
show_tree('/workspace')

## 3. 新しいモジュール構成のテスト

分割されたモジュールが正しく動作するかをテストします。

In [ ]:
# 新しいモジュールをインポートしてテスト
try:
    from src.eval import preprocessing, metrics, pipeline, cli
    print("✅ すべてのモジュールのインポートに成功")
except ImportError as e:
    print(f"❌ インポートエラー: {e}")

# 利用可能な関数を確認
print("\n=== preprocessing.py ===")
for name in dir(preprocessing):
    if not name.startswith('_'):
        print(f"- {name}")

print("\n=== metrics.py ===")
for name in dir(metrics):
    if not name.startswith('_'):
        print(f"- {name}")

print("\n=== pipeline.py ===")
for name in dir(pipeline):
    if not name.startswith('_'):
        print(f"- {name}")

## 4. サンプル画像での動作確認

実際のサンプル画像を使って、リファクタリング後のコードが正しく動作するかを確認します。

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# サンプル画像のパスを確認
data_dir = Path('/workspace/data/samples')
image_files = list(data_dir.glob('*.jpg'))
print("利用可能な画像ファイル:")
for img_path in image_files:
    print(f"- {img_path.name}")

# 画像を表示する関数
def show_images(images, titles, figsize=(15, 5)):
    fig, axes = plt.subplots(1, len(images), figsize=figsize)
    if len(images) == 1:
        axes = [axes]
    
    for i, (img, title) in enumerate(zip(images, titles)):
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(title)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# サンプル画像で前処理をテスト
if image_files:
    ref_path = None
    user_path = None
    
    for img_path in image_files:
        if 'ref_' in img_path.name:
            ref_path = img_path
        elif 'user_' in img_path.name:
            user_path = img_path
    
    if ref_path and user_path:
        print(f"お手本: {ref_path.name}")
        print(f"ユーザー: {user_path.name}")
        
        try:
            # 前処理を実行
            ref_img, ref_mask = preprocessing.preprocess(ref_path, size=256, dbg=False)
            user_img, user_mask = preprocessing.preprocess(user_path, size=256, dbg=False)
            
            # 結果を表示
            show_images(
                [ref_img, ref_mask*255, user_img, user_mask*255],
                ['お手本(グレー)', 'お手本(マスク)', 'ユーザー(グレー)', 'ユーザー(マスク)']
            )
            
            print("✅ 前処理が正常に完了しました")
            
        except Exception as e:
            print(f"❌ 前処理エラー: {e}")
    else:
        print("適切なサンプル画像が見つかりません")
else:
    print("サンプル画像が見つかりません")

## 5. 評価スコアの計算とパラメータ調整

各評価軸のスコアを計算し、パラメータの影響を確認します。

In [ ]:
# 評価スコアを計算
if 'ref_mask' in locals() and 'user_mask' in locals():
    # 各軸の個別スコアを計算
    shape_sc = metrics.shape_score(ref_mask, user_mask)
    
    cv_ref = metrics.stroke_cv(ref_mask)
    cv_user = metrics.stroke_cv(user_mask)
    black_sc = metrics.black_score(cv_ref, cv_user)
    
    ratio_ref = metrics.black_ratio(ref_mask)
    ratio_user = metrics.black_ratio(user_mask)
    white_sc = metrics.white_score(ratio_ref, ratio_user)
    
    center_sc = metrics.center_score(user_mask)
    
    print("=== 個別スコア ===")
    print(f"形スコア (IoU): {shape_sc:.3f}")
    print(f"黒スコア (線幅): {black_sc:.3f}")
    print(f"白スコア (密度): {white_sc:.3f}")
    print(f"場スコア (重心): {center_sc:.3f}")
    
    print("\n=== 詳細情報 ===")
    print(f"お手本の変動係数: {cv_ref:.3f}" if cv_ref else "お手本の変動係数: None")
    print(f"ユーザーの変動係数: {cv_user:.3f}" if cv_user else "ユーザーの変動係数: None")
    print(f"お手本の黒画素割合: {ratio_ref:.3f}")
    print(f"ユーザーの黒画素割合: {ratio_user:.3f}")
    
    # 総合評価
    scores = pipeline.evaluate_pair(ref_img, ref_mask, user_img, user_mask)
    print("\n=== 総合評価結果 ===")
    for key, value in scores.items():
        print(f"{key}: {value}")

## 6. パラメータ感度分析

重みパラメータを変更した場合の総合スコアへの影響を分析します。

In [ ]:
# パラメータ感度分析
if 'shape_sc' in locals():
    import matplotlib.pyplot as plt
    
    # 現在の重み
    current_weights = {
        "形": 0.30,
        "黒": 0.20, 
        "白": 0.30,
        "場": 0.20
    }
    
    individual_scores = {
        "形": shape_sc,
        "黒": black_sc,
        "白": white_sc,
        "場": center_sc
    }
    
    # 各軸の重みを変化させた場合の総合スコア
    weight_variations = np.linspace(0.0, 0.6, 13)
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    for i, (axis_name, base_weight) in enumerate(current_weights.items()):
        total_scores = []
        
        for weight in weight_variations:
            # 他の重みを調整して合計を1にする
            remaining = 1.0 - weight
            other_weights = {k: v for k, v in current_weights.items() if k != axis_name}
            total_other = sum(other_weights.values())
            
            if total_other > 0:
                adjusted_weights = {k: v * remaining / total_other for k, v in other_weights.items()}
            else:
                adjusted_weights = {k: remaining / 3 for k in other_weights.keys()}
            
            adjusted_weights[axis_name] = weight
            
            # 総合スコア計算
            total = sum(adjusted_weights[k] * individual_scores[k] for k in adjusted_weights.keys())
            total_scores.append(total * 100)
        
        axes[i].plot(weight_variations, total_scores, 'o-')
        axes[i].axvline(base_weight, color='red', linestyle='--', alpha=0.7, label=f'現在値: {base_weight}')
        axes[i].set_xlabel(f'{axis_name}軸の重み')
        axes[i].set_ylabel('総合スコア')
        axes[i].set_title(f'{axis_name}軸重みの影響')
        axes[i].grid(True, alpha=0.3)
        axes[i].legend()
    
    plt.tight_layout()
    plt.show()
    
    print("各軸の重みを変化させた場合の総合スコアへの影響を可視化しました。")

## 7. CLIインターフェースのテスト

新しいCLIモジュールが正しく動作するかをテストします。

In [ ]:
# CLIの動作テスト（実際のコマンド実行はしない）
if ref_path and user_path:
    print("=== CLI使用例 ===")
    print(f"python -m src.eval.cli {ref_path} {user_path}")
    print(f"python -m src.eval.cli {ref_path} {user_path} --json")
    print(f"python -m src.eval.cli {ref_path} {user_path} -s 512 --dbg")
    
    # 直接関数を呼び出してテスト
    print("\n=== 直接関数呼び出し結果 ===")
    try:
        import sys
        from io import StringIO
        
        # 引数を模擬
        original_argv = sys.argv
        sys.argv = ['cli.py', str(ref_path), str(user_path), '--json']
        
        # 標準出力をキャプチャ
        old_stdout = sys.stdout
        sys.stdout = captured_output = StringIO()
        
        try:
            # CLIメイン関数を実行
            result = cli.main()
            output = captured_output.getvalue()
            print(f"終了コード: {result}")
            print(f"出力:\n{output}")
        finally:
            sys.stdout = old_stdout
            sys.argv = original_argv
            
    except Exception as e:
        print(f"CLIテストエラー: {e}")
else:
    print("サンプル画像が不足しているため、CLIテストをスキップします")

## 8. まとめと今後の改善点

リファクタリングの結果をまとめ、今後の改善点を整理します。

In [ ]:
print("=== リファクタリング完了レポート ===")
print()
print("✅ 完了した作業:")
print("  - test.py を src/eval/ 以下に機能別モジュールに分割")
print("  - preprocessing.py: 画像前処理機能")
print("  - metrics.py: 4軸評価スコア計算")
print("  - pipeline.py: 評価パイプライン")
print("  - cli.py: コマンドラインインターフェース")
print("  - テストコード作成 (tests/ 以下)")
print("  - 適切なディレクトリ構成の実装")
print()
print("🔧 今後の改善点:")
print("  - より多くのサンプル画像でのテスト")
print("  - パラメータ最適化の自動化")
print("  - エラーハンドリングの強化")
print("  - ドキュメント化の充実")
print("  - CI/CDパイプラインの追加")
print()
print("📊 評価精度向上のアイデア:")
print("  - 深層学習による特徴抽出の導入")
print("  - より細かい線質評価指標")
print("  - 文字種別ごとの重み調整")
print("  - 複数のお手本との比較機能")

# プロジェクト統計
import os
total_lines = 0
py_files = 0

for root, dirs, files in os.walk('/workspace/src'):
    for file in files:
        if file.endswith('.py'):
            py_files += 1
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                total_lines += len(f.readlines())

print(f"\n📈 プロジェクト統計:")
print(f"  - Pythonファイル数: {py_files}")
print(f"  - 総行数: {total_lines}")
print(f"  - 平均ファイルサイズ: {total_lines//py_files if py_files > 0 else 0} 行")